In [1]:
import subprocess
import sys
import os
import shutil
import pandas as pd

In [2]:
def shell_do(command, log=False, return_log=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    res=subprocess.run(command.split(), stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return(res.stdout.decode('utf-8'))

In [3]:
# set paths
basedir = '/data/songy4/twas'
datadir = f'{basedir}/data_folder'
fusiondir = f'{basedir}/fusion_twas'
geno_path = f'{datadir}/qc_genotypes_twas'
gene_list_path = f'{datadir}/gene_list.txt'
pheno_path = f'{datadir}/expression_matrix_final.txt'
coord_path = f'{datadir}/twas_coordinate.txt'
covar_path = f'{datadir}/covariates_case.txt'

# Liftover genotype from hg38 to hg19 to match 1kg ref

In [4]:
bim = pd.read_csv(f'{geno_path}.bim', sep='\t', header=None, names=['chr', 'rsid', 'kb', 'pos', 'a1', 'a2'])
bim

,chr,rsid,kb,pos,a1,a2
0,1,rs145427775,0,10291,T,C
1,1,rs55998931,0,10492,T,C
2,1,rs199896944,0,13504,A,G
3,1,rs199856693,0,14933,A,G
4,1,rs201855936,0,14948,A,G
...,...,...,...,...,...,...
23858194,24,rs375378036,0,56887099,T,C
23858195,24,rs113496864,0,56887221,T,C
23858196,24,rs77686620,0,56887583,A,G
23858197,24,rs376130607,0,56887631,T,C


In [5]:
# get chrN:start-end positions for liftover of genotype from hg38 to hg19 to match 1kG LD ref
lift_outname = f'{datadir}/geno_hg38_positions.bed'
bim = pd.read_csv(f'{geno_path}.bim', sep='\t', header=None)
bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
bim['chr'] = 'chr' + bim['chr'].astype('str')
bim['end'] = bim['pos'] + 1
lift_out = bim[['chr', 'pos', 'end', 'rsid' ]].copy()
lift_out.to_csv(lift_outname, sep='\t', header=False, index=False)

PermissionError: [Errno 13] Permission denied: '/data/songy4/twas/data_folder/geno_hg38_positions.bed'

In [12]:
# pull liftOver from UCSC
#!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/liftOver -P /data/songy4/twas/liftover/
#!wget --timestamping 'ftp://hgdownload.cse.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz' -O /data/songy4/twas/liftover/hg38ToHg19.over.chain.gz
#!chmod +x /data/songy4/twas/liftover/liftOver

--2021-04-01 21:44:51--  http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/liftOver
Resolving dtn06-e0 (dtn06-e0)... 10.1.200.242
Connecting to dtn06-e0 (dtn06-e0)|10.1.200.242|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 37464040 (36M)
Saving to: '/data/songy4/twas/liftover/liftOver.1'

100%[======================================>] 37,464,040  22.1MB/s   in 1.6s   

2021-04-01 21:44:53 (22.1 MB/s) - '/data/songy4/twas/liftover/liftOver.1' saved [37464040/37464040]

for details.

--2021-04-01 21:44:53--  ftp://hgdownload.cse.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz
Resolving dtn06-e0 (dtn06-e0)... 10.1.200.242
Connecting to dtn06-e0 (dtn06-e0)|10.1.200.242|:3128... connected.
Proxy request sent, awaiting response... 200 Gatewaying
Length: 1246411 (1.2M) [text/plain]
Saving to: '/data/songy4/twas/liftover/hg38ToHg19.over.chain.gz'

100%[======================================>] 1,246,411   2.45MB/s   in 0.5s   

2021-04-01 21:44:54 (2.

In [15]:
# create command for liftover
liftOver = f'{basedir}/liftover/liftOver'
chainfile = f'{basedir}/liftover/hg38ToHg19.over.chain.gz'
liftover_output = f'{basedir}/geno_hg19_positions.bed'
unlifted = f'{basedir}/unlifted.bed'

liftover_cmd = f'\
{liftOver} {lift_outname} {chainfile} {liftover_output} {unlifted}'

shell_do(liftover_cmd)
liftover_cmd

Executing: /data/songy4/twas/liftover/liftOver /data/songy4/twas/data_folder/twas_geno_hg38_positions.bed /data/songy4/twas/liftover/hg38ToHg19.over.chain.gz /data/songy4/twas/twas_geno_hg19_positions.bed /data/songy4/twas/twas_unlifted.bed


'/data/songy4/twas/liftover/liftOver /data/songy4/twas/data_folder/twas_geno_hg38_positions.bed /data/songy4/twas/liftover/hg38ToHg19.over.chain.gz /data/songy4/twas/twas_geno_hg19_positions.bed /data/songy4/twas/twas_unlifted.bed'

In [16]:
# create map update file
lifted = pd.read_csv(liftover_output, sep='\t', header=None, names=['chr', 'pos', 'end', 'rsid' ])
print(lifted)
bim = pd.read_csv(f'{geno_path}.bim', sep='\t', header=None, names=['chr', 'rsid', 'kb', 'pos', 'a1', 'a2'])
print(bim)
bim_lift_merge = bim.merge(lifted, how='right', on='rsid')
lifted_bim = bim_lift_merge[['chr_x', 'rsid', 'kb', 'pos_y', 'a1', 'a2']].copy()
lifted_bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
print(lifted_bim)

            chr       pos       end         rsid
0          chr1     10291     10292  rs145427775
1          chr1     10492     10493   rs55998931
2          chr1     13504     13505  rs199896944
3          chr1     14933     14934  rs199856693
4          chr1     14948     14949  rs201855936
...         ...       ...       ...          ...
23060229  chr22  51239721  51239722  rs374333198
23060230  chr22  51239861  51239862  rs367873634
23060231  chr22  51239953  51239954  rs372413129
23060232  chr22  51240820  51240821  rs202228854
23060233  chr22  51242613  51242614  rs140611932

[23060234 rows x 4 columns]
          chr         rsid  kb       pos a1 a2
0           1  rs145427775   0     10291  T  C
1           1   rs55998931   0     10492  T  C
2           1  rs199896944   0     13504  A  G
3           1  rs199856693   0     14933  A  G
4           1  rs201855936   0     14948  A  G
...       ...          ...  ..       ... .. ..
23858194   24  rs375378036   0  56887099  T  C
2385819

In [ ]:
# use only lifted snps
lifted_bim['rsid'].to_csv(f'{geno_path}_hg19_lifted.snplist', sep='\t', header=False, index=False)

plink_extract_cmd = f'\
plink --bfile {geno_path}\
 --extract {geno_path}_hg19_lifted.snplist\
 --make-bed\
 --out {geno_path}_hg19_lifted'

shell_do(plink_extract_cmd)


In [ ]:
# move bim with old positions to new file
!mv {geno_path}_hg19_lifted.bim {geno_path}_hg19_lifted_old_positions.bim

In [ ]:
# write lifted bim to _hg19_lifted genotype name
lifted_bim.to_csv(f'{geno_path}_hg19_lifted.bim', sep='\t', header=False, index=False)

In [5]:
#check number of IDs in original .bim file
!cat {datadir}/qc_genotypes_twas_hg19_lifted.bim | wc -l

#look at the bim file (variance, chromosome and position, allele1 and allele2)
! head {datadir}/qc_genotypes_twas_hg19_lifted.bim

23060234
1	rs145427775	0	10291	T	C
1	rs55998931	0	10492	T	C
1	rs199896944	0	13504	A	G
1	rs199856693	0	14933	A	G
1	rs201855936	0	14948	A	G
1	rs71252251	0	14976	A	G
1	rs201045431	0	15029	A	G
1	rs368345873	0	15208	A	G
1	rs374029747	0	15774	A	G
1	rs201330479	0	16792	A	G


Liftover control genotype from hg38 to hg19

In [9]:
# set paths for controls
control_geno_path = f'{datadir}/qc_genotypes_twas_control'
pheno_path = f'{datadir}/twas_expression_control.txt'
covar_path = f'{datadir}/covariates_control.txt'

In [10]:
# get chrN:start-end positions for liftover of genotype from hg38 to hg19 to match 1kG LD ref
lift_outname_cont = f'{datadir}/control_geno_hg38_positions.bed'
bim_cont = pd.read_csv(f'{geno_path}_control.bim', sep='\t', header=None)
bim_cont.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
bim_cont['chr'] = 'chr' + bim_cont['chr'].astype('str')
bim_cont['end'] = bim_cont['pos'] + 1
lift_out_cont = bim_cont[['chr', 'pos', 'end', 'rsid' ]].copy()
lift_out_cont.to_csv(lift_outname_cont, sep='\t', header=False, index=False)

In [14]:
# create command for liftover
liftOver = f'{basedir}/liftover/liftOver'
chainfile = f'{basedir}/liftover/hg38ToHg19.over.chain.gz'
liftover_output_cont = f'{basedir}/control_geno_hg19_positions.bed'
unlifted_cont = f'{basedir}/control_unlifted.bed'

liftover_cont_cmd = f'\
{liftOver} {lift_outname_cont} {chainfile} {liftover_output_cont} {unlifted_cont}'

shell_do(liftover_cont_cmd)
liftover_cont_cmd

Executing: /data/songy4/twas/liftover/liftOver /data/songy4/twas/data_folder/control_geno_hg38_positions.bed /data/songy4/twas/liftover/hg38ToHg19.over.chain.gz /data/songy4/twas/control_geno_hg19_positions.bed /data/songy4/twas/control_unlifted.bed


'/data/songy4/twas/liftover/liftOver /data/songy4/twas/data_folder/control_geno_hg38_positions.bed /data/songy4/twas/liftover/hg38ToHg19.over.chain.gz /data/songy4/twas/control_geno_hg19_positions.bed /data/songy4/twas/control_unlifted.bed'

In [15]:
# create map update file
lifted_cont = pd.read_csv(liftover_output_cont, sep='\t', header=None, names=['chr', 'pos', 'end', 'rsid' ])

bim_cont = pd.read_csv(f'{geno_path}_control.bim', sep='\t', header=None, names=['chr', 'rsid', 'kb', 'pos', 'a1', 'a2'])
bim_lift_merge_cont = bim_cont.merge(lifted_cont, how='right', on='rsid')
lifted_bim_cont = bim_lift_merge_cont[['chr_x', 'rsid', 'kb', 'pos_y', 'a1', 'a2']].copy()
lifted_bim_cont.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']

In [16]:
# use only lifted snps
lifted_bim_cont['rsid'].to_csv(f'{geno_path}_control_hg19_lifted.snplist', sep='\t', header=False, index=False)

plink_extract_cont_cmd = f'\
plink --bfile {geno_path}_control\
 --extract {geno_path}_control_hg19_lifted.snplist\
 --make-bed\
 --out {geno_path}_control_hg19_lifted'

shell_do(plink_extract_cont_cmd)

Executing: plink --bfile /data/songy4/twas/data_folder/qc_genotypes_twas_control --extract /data/songy4/twas/data_folder/qc_genotypes_twas_control_hg19_lifted.snplist --make-bed --out /data/songy4/twas/data_folder/qc_genotypes_twas_control_hg19_lifted


In [17]:
# move bim with old positions to new file
!mv {geno_path}_control_hg19_lifted.bim {geno_path}_control_hg19_lifted_old_positions.bim

In [19]:
# write lifted bim to _hg19_lifted genotype name
lifted_bim_cont.to_csv(f'{geno_path}_control_hg19_lifted.bim', sep='\t', header=False, index=False)

In [20]:
#check number of IDs in original .bim file
!cat {datadir}/qc_genotypes_twas_control_hg19_lifted.bim | wc -l

#look at the bim file (variance, chromosome and position, allele1 and allele2)
! head {datadir}/qc_genotypes_twas_control_hg19_lifted.bim

23060234
1	rs145427775	0	10291	T	C
1	rs55998931	0	10492	T	C
1	rs199896944	0	13504	A	G
1	rs199856693	0	14933	A	G
1	rs201855936	0	14948	A	G
1	rs71252251	0	14976	A	G
1	rs201045431	0	15029	A	G
1	rs368345873	0	15208	A	G
1	rs374029747	0	15774	A	G
1	rs201330479	0	16792	A	G
